IMPORTING LIBRARIES 

In [1]:
# calling all the libraries 
!pip install ipykernel
!pip install ctransformers
from langchain.prompts import PromptTemplate
from langchain.chains import retrieval_qa 
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.vectorstores import pinecone
#import pinecone
from langchain.document_loaders import PyMuPDFLoader, DirectoryLoader, PyPDFLoader
from langchain.llms import CTransformers
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
import os
PINECONE_API_KEY = "d43c6d43-4465-4079-a3b6-a3cfc22bc808"
PINECONE_API_ENV = "us-east-1"
os.environ['PINECONE_API_KEY'] = "d43c6d43-4465-4079-a3b6-a3cfc22bc808"


DATA EXTRACTION

In [3]:
#extrating data from pdf

def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents




In [4]:
extracter_data = load_pdf(r"C:\Users\harshita\proj2\data")

CHUNKS 

In [5]:
#converting corpus into chunks 
def text_split(extracter_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500 , 
                                   chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracter_data)

    return text_chunks

In [6]:
text_chunks=text_split(extracter_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 7020


HUGGING FACE 



In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

In [8]:
#vector embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name = 'sentence-transformers/all-MiniLM-L6-v2'
    )
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

C:\Users\harshita\AppData\Local\Temp\ipykernel_15272\3523299516.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(
c:\Users\harshita\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
query_results = embeddings.embed_query("Hello world")
print(len(query_results))

384


In [ ]:
# Generate embeddings for all text chunks
texts = [chunk.page_content for chunk in text_chunks]
embeddings_list = embeddings.embed_documents(texts)

In [ ]:
import uuid
#Prepare vectors for upsert
vectors = []
for i, (embedding, chunk) in enumerate(zip(embeddings_list, text_chunks)):
    vector_id = f"chunk-{i}-{uuid.uuid4()}"  # Unique ID
    metadata = {
        "text": chunk.page_content, 
        #"source": chunk.metadata.get("source", "unknown"),
        #"page": chunk.metadata.get("page", "unknown"),
        # Add more metadata fields as needed
    }
    vectors.append((vector_id, embedding, metadata))

In [ ]:
len(vectors)

In [11]:
from dotenv import load_dotenv
load_dotenv()

False

PINECONE INTEGRATION


In [12]:
PINECONE_API_KEY=os.environ.get('d43c6d43-4465-4079-a3b6-a3cfc22bc808')
#OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [13]:
#to create a new index
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

In [ ]:
#already created index
#to create a new index
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"
index = pc.Index(index_name)

In [ ]:
#Define a batch upsert function
def batch_upsert(index, vectors, batch_size=500):
    """
    Upsert vectors into Pinecone index in batches.

    :param index: Pinecone Index object
    :param vectors: List of tuples (id, vector, metadata)
    :param batch_size: Number of vectors per batch
    """
    total_vectors = len(vectors)
    for i in range(0, total_vectors, batch_size):
        batch = vectors[i:i + batch_size]
        try:
            upsert_response = index.upsert(vectors=batch)
            print(f"Upserted batch {i // batch_size + 1} ({len(batch)} vectors).")
        except pinecone.core.client.exceptions.PineconeException as e:
            print(f"Error upserting batch {i // batch_size + 1}: {e}")

# Performing batched upsert
batch_size = 500  
batch_upsert(index, vectors, batch_size=batch_size)

Semantic search function

In [ ]:
# Semantic search function
def semantic_search(query, top_k=3):
    # Generate embedding for the query
    query_embedding = embeddings.embed_query(query)
    
    # Query the Pinecone index
    search_results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )
    
    return search_results

In [ ]:
#Example usage of semantic search
if __name__ == "__main__":
    # Get user input for the search query
    user_query = input("Enter your Question ")
    
    # Perform semantic search
    results = semantic_search(user_query, top_k=3)
    

# Display the results
print(results.matches)

PROMPTING

In [14]:
prompt_template = """
Use the following peices of interaction to answer the users question.
If you dont know the abswer , just say u dont know, dont make up your own answers.

context : {context}
question : {question}

only return the helpful answer below

helpful answer: 

"""

In [15]:
Prompt = PromptTemplate(
    input_variables=["context","question"],
    template = prompt_template
)

chain_type = {"prompt" : Prompt}

In [25]:
llm = CTransformers(model = "MODEL/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

VECTOR SEARCH

In [17]:
from langchain.vectorstores import Pinecone

docsearch = Pinecone.from_documents(
    documents = text_chunks,
    index_name = index_name,
    embedding =embeddings 
)

In [18]:
docsearch

DEFINING RETRIEVER

In [19]:
retriever = docsearch.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":3}
)

In [20]:
retriever_docs = retriever.invoke("What is Acne?")

In [21]:
retriever_docs

[Document(metadata={'page': 37.0, 'source': 'C:\\Users\\harshita\\proj2\\data\\Medical_book.pdf'}, page_content='Acidosis seeRespiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when thepores of the skin become clogged with oil, dead skincells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne, is'),
 Document(metadata={'page': 38.0, 'source': 'C:\\Users\\harshita\\proj2\\data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25Acne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceousglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)GEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(metadata={'page': 239.0, 'source': 'C:\\Users\\harshita\\proj2\\data\\Medical_book.pdf'}, page_content

In [26]:
qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = docsearch.as_retriever(search_kwargs = {'k':2}),
    return_source_documents = True,
    #chain_type_kwargs = chain_type_kwargs
    )

In [ ]:
while True:
    user_input = input(f"Input Prompt")
    result = qa({"query":user_input})
    print("Response : ", result["result"])

Response :   Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
